# Training a neural network on EMNIST with Keras



In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

### Load a dataset

Load the EMNIST letters dataset with the following arguments:

* `shuffle_files=True`: The EMNIST data is only stored in a single file, but for larger datasets with multiple files on disk, it's good practice to shuffle them when training.
* `as_supervised=True`: Returns a tuple `(img, label)` instead of a dictionary `{'image': img, 'label': label}`.

In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'emnist/letters',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

### Build a training pipeline

Apply the following transformations:

* `tf.data.Dataset.map`: TFDS provide images of type `tf.uint8`, while the model expects `tf.float32`. Therefore, you need to normalize images.
* `tf.data.Dataset.cache` As you fit the dataset in memory, cache it before shuffling for a better performance.<br/>
__Note:__ Random transformations should be applied after caching.
* `tf.data.Dataset.shuffle`: For true randomness, set the shuffle buffer to the full dataset size.<br/>
__Note:__ For large datasets that can't fit in memory, use `buffer_size=1000` if your system allows it.
* `tf.data.Dataset.batch`: Batch elements of the dataset after shuffling to get unique batches at each epoch.
* `tf.data.Dataset.prefetch`: It is good practice to end the pipeline by prefetching [for performance](https://www.tensorflow.org/guide/data_performance#prefetching).

In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

## Step 2: Create and train the model

Plug the TFDS input pipeline into a simple Keras model, compile the model, and train it.

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(27)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
)

In [ ]:
from tensorflow.keras import datasets, layers, models 

model = models.Sequential()
# 第1层卷积，卷积核大小为3*3，32个，28*28为待训练图片的大小
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
# 第2层卷积，卷积核大小为3*3，64个
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
# 第3层卷积，卷积核大小为3*3，64个
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(27, activation='softmax'))


model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_test,
)

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os

'''
python 3.7
tensorflow 2.0.0b0
pillow(PIL) 4.3.0
'''


def predict(image_value, image_path):
    # 1. invert the image horizontally and rotated 90 anti-clockwise
    # 2. resize the image as size of (28 * 28)
    img = Image.open(image_path)
    img = img.transpose(Image.FLIP_LEFT_RIGHT).transpose(Image.ROTATE_90)
    img = img.resize((28, 28), Image.ANTIALIAS).convert('L')
    img = np.reshape(img, (28, 28, 1)) / 255.
    x = np.array([1 - img])

    y = model.predict(x)

    # np.argmax() is the predict label. 
    # Here, A->1, B->2, ... , Z->26, so I transfrom label to char.
    print('{} ->  Predict {}, {}'.format(image_value,chr(np.argmax(y[0]) + 64),image_value == chr(np.argmax(y[0]) + 64)))
    return image_value == chr(np.argmax(y[0]) + 64)


def testPredict(file_path):
  count = 0
  num = 0
  for filename in os.listdir(file_path):
    if filename.endswith('png'):
      num += 1
      value = filename.split('.')[0]
      res = predict(value,os.path.join(file_path,filename))
      if not res:
        count += 1
  
  print('acc:',1-count/num)
# print(res0)

In [ ]:
testPredict('/content/test')

In [ ]:
path = '/content/sample_data/B.png'
img = Image.open(path).transpose(Image.FLIP_LEFT_RIGHT).transpose(Image.ROTATE_90)
img